# 1. NLTK Initialization and Setup:

In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import os
import string
import logging
import re
from collections import defaultdict, Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Initialize stopwords and lemmatizer
STOPWORDS = set(stopwords.words('english'))
LEMMATIZER = WordNetLemmatizer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


# 2. Loading Text Files:

In [2]:
def load_text_files(folder_path):
    """ Reads all .txt files in a folder and returns a dictionary with filenames as keys and content as values. """
    data = {}
    doc_id_to_filename = {}
    doc_id = 0  # Initialize document ID
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                data[doc_id] = file.read()
            doc_id_to_filename[doc_id] = filename  # Map doc_id to filename
            logging.info(f"Loaded file: {filename} with doc_id: {doc_id}")
            doc_id += 1  # Increment document ID for the next file
    return data, doc_id_to_filename

# Example folder path: Replace this with your folder path
folder_path = "/content/drive/MyDrive/Information Retrieval System/Week2/docs"
data, doc_id_to_filename = load_text_files(folder_path)


# 3. Text Cleaning Pipeline:

In [3]:
def clean_text(text):
    """Performs text cleaning: removing special characters, tokenization, stop word removal, and lemmatization."""
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and punctuation
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    # Tokenize the cleaned text
    tokens = word_tokenize(text)
    # Remove stopwords and lemmatize
    cleaned_tokens = [LEMMATIZER.lemmatize(word) for word in tokens if word not in STOPWORDS]
    return cleaned_tokens

# Test cleaning function with a sample text
print(clean_text(data[0]))


['applesupport', 'many', 'user', 'reporting', 'issue', 'latest', 'io', 'update', 'battery', 'life', 'seems', 'drain', 'faster', 'apps', 'like', 'safari', 'crashing', 'frequently', 'user', 'tried', 'reinstalling', 'update', 'doesnt', 'seem', 'resolve', 'problem', 'check', 'discussion', 'httpssupportapplecomiosupdates', 'iosupdate', 'appleproblems', '105836', 'working', 'ok', 'miriam', 'link', 'help', 'httpstco0m2mph15eh', 'mm', 'virgintrains', 'still', 'havent', 'heard', 'amp', 'number', 'im', 'directed', 'phone', 'dead', 'end', 'amp', 'live', 'chat', 'doesnt', 'work', 'someone', 'call', '105836', 'thats', 'miriam', 'team', 'send', 'email', 'shortly', 'hp', '105837', 'help', 'version', 'io', 'find', 'setting', 'gt', 'general', 'gt', 'reply', 'dm', 'httpstcogdrqu22ypt', '105838', 'applesupport', 'suffering', 'hope', 'find', 'solution', 'applesupport', 'hi', 'apple', 'ive', 'concern', 'latest', 'io', 'slow', 'iphone6', 'happy', 'solution', 'please', '105839', 'thanks', 'reaching', 'u', 'a

# 4. Inverted Index Construction

In [5]:
def build_inverted_index(data):
    """Builds an inverted index from the cleaned text data and tracks term frequencies."""
    inverted_index = defaultdict(set)
    term_frequencies = Counter()  # Track the frequency of each term
    for doc_id, content in data.items():
        cleaned_tokens = clean_text(content)
        for token in cleaned_tokens:
            inverted_index[token].add(doc_id)
            term_frequencies[token] += 1  # Update term frequency
    return inverted_index, term_frequencies

inverted_index, term_frequencies = build_inverted_index(data)


# 5. Boolean Query Processing: AND, OR, and NOT Operations:






In [4]:
def boolean_query(query, inverted_index, doc_id_to_filename):
    """Processes a Boolean query and returns the matching documents."""
    tokens = query.lower().split()
    result_set = set()

    if 'and' in tokens:
        terms = [term for term in tokens if term not in ['and', 'or', 'not']]
        result_set = inverted_index[terms[0]]
        for term in terms[1:]:
            if term in inverted_index:
                result_set = result_set.intersection(inverted_index[term])

    elif 'or' in tokens:
        terms = [term for term in tokens if term not in ['and', 'or', 'not']]
        for term in terms:
            if term in inverted_index:
                if not result_set:
                    result_set = inverted_index[term]
                else:
                    result_set = result_set.union(inverted_index[term])

    elif 'not' in tokens:
        term = tokens[1]
        if term in inverted_index:
            result_set = set(inverted_index.keys()) - inverted_index[term]
        else:
            result_set = set(inverted_index.keys())

    else:
        if query in inverted_index:
            result_set = inverted_index[query]
        else:
            result_set = set()

    # Convert doc_ids to filenames
    result_filenames = [doc_id_to_filename[doc_id] for doc_id in result_set if doc_id in doc_id_to_filename]
    logging.info(f"Query '{query}' resulted in: {result_filenames}")
    return result_filenames


# 6. Main Function:

In [13]:
def write_results_to_file(queries, inverted_index, doc_id_to_filename, output_file_path):
    """Writes the query results to a text file at the specified location."""
    with open(output_file_path, "w") as result_file:
        for query in queries:
            result = boolean_query(query, inverted_index, doc_id_to_filename)
            result_str = f"Results for '{query}': {result}\n"
            print(result_str)  # Print to console
            result_file.write(result_str)  # Write to file


In [12]:
def main():
    # Load dataset
    folder_path = "/content/drive/MyDrive/Information Retrieval System/Week2/docs"
    data, doc_id_to_filename = load_text_files(folder_path)

    # Build the inverted index and term frequencies
    inverted_index, term_frequencies = build_inverted_index(data)

    # Example queries
    queries = [
        "Netflix AND Xbox",
        "Microsoft OR Windows",
        "NOT Xbox",
        "Xbox",
        "Windows AND NOT Netflix",
        "Spotify AND Bluetooth",
        "Apple OR Battery",
        "Virgin AND Train",
        "Telemarketing AND O2",
        "Spotify AND Bluetooth AND NOT Car",
        "Battery AND Drain",
        "Netflix OR Xbox",
        "Spotify AND NOT Bluetooth",
        "Windows OR Xbox",
        "British AND Flight",
        "Delayed AND Train",
        "NOT Spotify",
        "Flight AND Delay AND NOT Train",
        "Safari AND Crash",
        "iPhone AND NOT Telemarketing",
        "Windows AND WiFi AND Freeze",
        "Spotify AND Pause"
    ]

     # Specifying the output folder path for saving query results
    output_folder_path = "/content/drive/MyDrive/Information Retrieval System/Week2/docs"

     # Create the output file path
    output_file_path = os.path.join(output_folder_path, "query_results.txt")

    # Write query results to the specified output file path
    write_results_to_file(queries, inverted_index, doc_id_to_filename, output_file_path)

if __name__ == "__main__":
    main()


Results for 'Netflix AND Xbox': ['query_results.txt']

Results for 'Microsoft OR Windows': ['Docs5.txt', 'query_results.txt']

Results for 'NOT Xbox': []

Results for 'Xbox': []

Results for 'Windows AND NOT Netflix': []

Results for 'Spotify AND Bluetooth': ['query_results.txt']

Results for 'Apple OR Battery': ['Docs1.txt', 'query_results.txt']

Results for 'Virgin AND Train': ['query_results.txt']

Results for 'Telemarketing AND O2': ['query_results.txt']

Results for 'Spotify AND Bluetooth AND NOT Car': ['query_results.txt']

Results for 'Battery AND Drain': ['Docs1.txt', 'query_results.txt']

Results for 'Netflix OR Xbox': ['Docs3.txt', 'Docs7.txt', 'query_results.txt']

Results for 'Spotify AND NOT Bluetooth': ['query_results.txt']

Results for 'Windows OR Xbox': ['Docs7.txt', 'query_results.txt']

Results for 'British AND Flight': ['query_results.txt']

Results for 'Delayed AND Train': ['query_results.txt']

Results for 'NOT Spotify': []

Results for 'Flight AND Delay AND NOT Tr